In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
import tensorflow as tf, keras
from keras import layers
from keras.layers import StringLookup, IntegerLookup, Embedding, Normalization, Dense

In [ ]:
path = os.path.normpath("C:\\Users\lenha\Dropbox\\baseball\\all_seasons_hitters.csv")

In [ ]:
df = pd.read_csv(path)
df = (
    df.rename(columns={"K%": "K", "BB%": "BB"})
    .assign(BB=lambda df: df['BB'].str.replace('%', ""), K=lambda df: df['K'].str.replace('%', ""))
    .astype({"BB": 'float', 'K': 'float'})
    .drop(columns=['xwOBA'])
)
df = df.merge(df.assign(Season=lambda df: df.Season + 1, target=lambda df: df.HR)[['Season', 'playerid','target']], on=['Season', 'playerid'], how='left').dropna()
df

In [ ]:
def make_encoders(df: pd.DataFrame, as_cat=set()) -> dict:
    encoders = {}
    for col in df:

        name=f"{col}_encoder"
        if df[col].dtype == 'object' or col in as_cat:
            # print(df[col])
            if isinstance(df[col][0], str):
                lookup_layer = keras.layers.StringLookup
                print(col, " is string type.")
            else:
                print(col, "is int type.")
                lookup_layer = keras.layers.IntegerLookup
            values =  df[col].unique().tolist()
            lookup = lookup_layer(vocabulary=values)
            embedding = keras.layers.Embedding(len(values), int(np.log(len(values))))
            # FIXME: just pass an input to the damned sequence.
            # and flatten
            print(lookup)
            flatten = layers.Flatten()
            encoder = keras.layers.Lambda(lambda x: flatten(embedding(lookup_layer(x))), name=name)
            encoders[col] = encoder
        else:
            print(col, " is numeric.")
            encoders[col] = keras.layers.Normalization(name=name)
        # encoders[col] = map_to_encoder(args)
    return encoders

def make_inputs(df, as_cat=set()) -> dict:
    encoders = make_encoders(df, as_cat)
    return {k: encoder(keras.Input(shape=(1,), name=k)) for k, encoder in encoders.items()}


In [ ]:
from sklearn.model_selection import train_test_split

def prepare_data(df: pd.DataFrame) -> tuple[pd.DataFrame, ...]:
    df = df.copy()
    test_size = int(min(len(df) / 5, 5_000))
    target = df.pop('target')
    split_data = train_test_split(df, target, test_size=test_size)
    return [dict(df) for df in splilt]

x_train, x_val, y_train, y_val = prepare_data(df.drop(columns='Name'))

In [ ]:
StringLookup(vocabulary=['SEA'])(['se', 'SEA'])

In [ ]:
inputs = make_inputs(x_train, as_cat=['playerid', 'Season'])
inputs

In [ ]:
list(inputs.values())

In [ ]:
x = layers.Concatenate()(list(inputs.values()))
out = Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, out)

In [ ]:
model.compile('adam', keras.losses.MeanSquaredError())

In [ ]:
x_train.values

In [ ]:
model.fit(x_train.values, y_train.values)

In [ ]:
for i in range(12):
    print(f"    {i}: ( ),")